In [ ]:
# Flight Management System Project
# Developed in Python by Krishnendu Mondal
# Libraries Used: pandas, numpy, matplotlib

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date
import os

# File paths
FLIGHTS_CSV = "flights.csv"
BOOKINGS_CSV = "bookings.csv"

# ===============================
# Initialize Data Files
# ===============================
def initialize_files():
    flight_cols = ['flight_id','airline','origin','destination','departure','arrival','capacity','fare']
    booking_cols = ['booking_id','flight_id','passenger_name','seat_no','status','booking_date','fare_paid']
    if not os.path.exists(FLIGHTS_CSV):
        pd.DataFrame(columns=flight_cols).to_csv(FLIGHTS_CSV, index=False)
    if not os.path.exists(BOOKINGS_CSV):
        pd.DataFrame(columns=booking_cols).to_csv(BOOKINGS_CSV, index=False)


# ===============================
# Add Flight
# ===============================
def add_flight():
    airline = input("Enter Airline Name: ")
    origin = input("Enter Origin City Code (e.g., DEL): ")
    destination = input("Enter Destination City Code (e.g., BLR): ")
    departure = input("Enter Departure (YYYY-MM-DD HH:MM): ")
    arrival = input("Enter Arrival (YYYY-MM-DD HH:MM): ")
    capacity = int(input("Enter Capacity: "))
    fare = float(input("Enter Fare: "))

    flights = pd.read_csv(FLIGHTS_CSV)
    fid = 2001 if flights.empty else int(flights['flight_id'].max()) + 1

    new = {
        'flight_id': fid,
        'airline': airline,
        'origin': origin,
        'destination': destination,
        'departure': departure,
        'arrival': arrival,
        'capacity': capacity,
        'fare': fare
    }
    flights = pd.concat([flights, pd.DataFrame([new])], ignore_index=True)
    flights.to_csv(FLIGHTS_CSV, index=False)
    print(f"\n Flight added successfully with Flight ID: {fid}")


# ===============================
# View Flights
# ===============================
def view_flights():
    flights = pd.read_csv(FLIGHTS_CSV)
    if flights.empty:
        print("\nNo flights available.")
    else:
        print("\n=== Available Flights ===")
        print(flights.to_string(index=False))


# ===============================
# Helper – Next Seat Number
# ===============================
def _next_seat(flight_id):
    bookings = pd.read_csv(BOOKINGS_CSV)
    flight_bookings = bookings[(bookings['flight_id']==flight_id) & (bookings['status']=='CONFIRMED')]
    if flight_bookings.empty:
        return 1
    else:
        return int(flight_bookings['seat_no'].max()) + 1


# ===============================
# Book Ticket
# ===============================
def book_ticket():
    try:
        flight_id = int(input("Enter Flight ID: "))
        passenger_name = input("Enter Passenger Name: ")

        flights = pd.read_csv(FLIGHTS_CSV)
        if flight_id not in flights['flight_id'].values:
            print("Flight ID not found.")
            return

        flight = flights[flights['flight_id']==flight_id].iloc[0]
        bookings = pd.read_csv(BOOKINGS_CSV)

        confirmed_count = bookings[(bookings['flight_id']==flight_id) & (bookings['status']=='CONFIRMED')].shape[0]

        if confirmed_count >= int(flight['capacity']):
            status = 'WAITLISTED'
            seat = None
            fare_paid = 0.0
        else:
            seat = _next_seat(flight_id)
            status = 'CONFIRMED'
            fare_paid = float(flight['fare'])

        booking_id = 3001 if bookings.empty else int(bookings['booking_id'].max()) + 1
        new_booking = {
            'booking_id': booking_id,
            'flight_id': flight_id,
            'passenger_name': passenger_name,
            'seat_no': seat,
            'status': status,
            'booking_date': date.today().strftime('%Y-%m-%d'),
            'fare_paid': fare_paid
        }

        bookings = pd.concat([bookings, pd.DataFrame([new_booking])], ignore_index=True)
        bookings.to_csv(BOOKINGS_CSV, index=False)

        print(f"\n Booking Successful! Booking ID: {booking_id}, Status: {status}")

    except Exception as e:
        print("Error:", e)


# ===============================
# Cancel Booking
# ===============================
def cancel_booking():
    try:
        booking_id = int(input("Enter Booking ID to cancel: "))
        bookings = pd.read_csv(BOOKINGS_CSV)

        if booking_id not in bookings['booking_id'].values:
            print("Booking ID not found.")
            return

        idx = bookings[bookings['booking_id']==booking_id].index[0]

        if bookings.at[idx,'status'] == 'CANCELLED':
            print("Already cancelled.")
            return

        bookings.at[idx,'status'] = 'CANCELLED'
        bookings.at[idx,'seat_no'] = None
        bookings.to_csv(BOOKINGS_CSV, index=False)

        flight_id = int(bookings.loc[idx,'flight_id'])
        waitlisted = bookings[(bookings['flight_id']==flight_id) & (bookings['status']=='WAITLISTED')]

        if not waitlisted.empty:
            first_idx = waitlisted.index[0]
            seat = _next_seat(flight_id)
            bookings.at[first_idx,'status'] = 'CONFIRMED'
            bookings.at[first_idx,'seat_no'] = seat

            flights = pd.read_csv(FLIGHTS_CSV)
            fare = float(flights[flights['flight_id']==flight_id].iloc[0]['fare'])
            bookings.at[first_idx,'fare_paid'] = fare

            bookings.to_csv(BOOKINGS_CSV, index=False)
            print(f"Booking cancelled. Waitlisted Booking ID {int(bookings.at[first_idx,'booking_id'])} promoted.")
        else:
            print("Booking cancelled successfully.")

    except Exception as e:
        print("Error:", e)


# ===============================
# View Bookings
# ===============================
def view_bookings():
    bookings = pd.read_csv(BOOKINGS_CSV)
    if bookings.empty:
        print("\nNo bookings found.")
    else:
        print("\n=== All Bookings ===")
        print(bookings.to_string(index=False))


# ===============================
# Occupancy Pie Chart
# ===============================
def occupancy_pie_chart():
    try:
        flight_id = int(input("Enter Flight ID for occupancy chart: "))
        bookings = pd.read_csv(BOOKINGS_CSV)
        flights = pd.read_csv(FLIGHTS_CSV)

        if flight_id not in flights['flight_id'].values:
            print("Flight ID not found.")
            return

        cap = int(flights[flights['flight_id']==flight_id].iloc[0]['capacity'])
        confirmed = bookings[(bookings['flight_id']==flight_id) & (bookings['status']=='CONFIRMED')].shape[0]
        waitlisted = bookings[(bookings['flight_id']==flight_id) & (bookings['status']=='WAITLISTED')].shape[0]
        cancelled = bookings[(bookings['flight_id']==flight_id) & (bookings['status']=='CANCELLED')].shape[0]
        available = max(cap - confirmed, 0)

        labels = ['Confirmed', 'Available', 'Waitlisted', 'Cancelled']
        sizes = [confirmed, available, waitlisted, cancelled]

        plt.figure(figsize=(6,6))
        plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
        plt.title(f"Occupancy for Flight {flight_id}")
        plt.show()

    except Exception as e:
        print("Error:", e)


# ===============================
# Revenue Histogram
# ===============================
def revenue_histogram():
    bookings = pd.read_csv(BOOKINGS_CSV)
    if bookings.empty:
        print("No bookings found.")
        return

    rev = bookings[bookings['status']=='CONFIRMED'].groupby('flight_id')['fare_paid'].sum().reset_index()
    plt.figure(figsize=(8,5))
    plt.bar(rev['flight_id'].astype(str), rev['fare_paid'])
    plt.xlabel("Flight ID")
    plt.ylabel("Total Revenue")
    plt.title("Revenue per Flight")
    plt.grid(True)
    plt.show()


# ===============================
# Main Menu
# ===============================
def menu():
    initialize_files()
    while True:
        print("\n==== Flight Management System ====")
        print("1. Add Flight")
        print("2. View Flights")
        print("3. Book Ticket")
        print("4. Cancel Booking")
        print("5. View Bookings")
        print("6. Generate Occupancy Pie Chart")
        print("7. Generate Revenue Histogram")
        print("8. Exit")

        choice = input("Enter your choice: ")

        if choice == '1':
            add_flight()
        elif choice == '2':
            view_flights()
        elif choice == '3':
            book_ticket()
        elif choice == '4':
            cancel_booking()
        elif choice == '5':
            view_bookings()
        elif choice == '6':
            occupancy_pie_chart()
        elif choice == '7':
            revenue_histogram()
        elif choice == '8':
            print("Exiting Flight Management System. Goodbye!")
            break
        else:
            print("Invalid choice. Please try again.")


# Run the menu
if __name__ == "__main__":
    menu()
